In [1]:
# imports
# Required libraries to be installed: Pillow, OpenCV
import cv2 as cv
import numpy as np
import glob
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display
import PIL.Image
import io

In [2]:
# helper functions
def show_image(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def read_image(index, source, resize):
    image = cv.imread(source[index])
    return cv.resize(image, (0, 0), fx=resize, fy=resize)

In [4]:
# I tried with multiple images in the beginning
task1_images = glob.glob(os.path.join('./', "training_data_Task1_*.jpg")) 
char_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}